# Lab Class Week 8

## Explicit Finite Difference

Let us solve the Black Scholes call option $V$ such that
$$
\frac{\partial V}{\partial t}+\frac12\sigma^2 S^2 \frac{\partial^2 V}{\partial S^2}+rS \frac{\partial V}{\partial S}
-rV=0
$$
with terminal condition
$$
V(S,T) = \max(S-X,0)
$$

For the explicit method we shall need:
- All parameters for the option, such as $X$ and $S_0$ etc.
- The number of divisions in stock, $jMax$, and divisions in time $iMax$
- The size of the divisions $\Delta S=S_{max}/jMax$ and $\Delta t = T/iMax$
- A vector to store the stock prices, and two to store the option values at the current and previous time level.

It will make things easier if you just declare these at the top of the program before you start doing anything with them.

Start with your include statements:

In [1]:
#include <iostream>
#include <fstream>
#include <cmath>
#include <vector>
#include <algorithm>
using namespace std;

- Now declare the parameters for the problem, local grid variables and the vectors required.
- Set the values of the parameters to the following: $\sigma=0.4$, $r=0.05$, $X=2$, $T=1$, and $iMax=jMax=4$.
- Note that in option pricing problems we must for $S$ over the semi inifinite domain, therefore numerically we need to choose an appropriate $S_{max}$. 
- In general, you should set this according to mathematical reasoning if you want your code to be generic and work in all cases. 
- Often we just choose $S_{max}=K\times X$ where $K$ is some constant (say 5) but it should really be linked to the probability distribution of the underlying asset. 
- For simplicity here we choose $S_{max}=2X$ so that small grids still work, for any decent level of accuracy $S_{max}$ will need to set much higher.
- Next assuming we have set the correct level of storage to your vector, calculate the values of $\Delta S$ and $\Delta t$ for your grid and using them to assign initial values to the stock price vector, and the option value vectors.

In [2]:
{
    // declare and initialise Black Scholes parameters
    double S0=100.,X=100.,T=1.,r=0.06,sigma=0.2;
    // declare and initialise grid paramaters 
    int iMax=4,jMax=4;
    // declare and initialise local variables (ds,dt)
    double S_max=2*X;
    double dS=S_max/jMax;
    double dt=T/iMax;
    // create storage for the stock price and option price (old and new)
    vector<double> S(jMax+1),vOld(jMax+1),vNew(jMax+1);
    // setup and initialise the stock price 
    for(int j=0;j<=jMax;j++)
    {
        S[j] = j*dS;
    }
    // setup and initialise the final conditions on the option price 
    for(int j=0;j<=jMax;j++)
    {
        vOld[j] = max(S[j]-X,0.);
        vNew[j] = max(S[j]-X,0.);
        cout << iMax << " " << j << " " << S[j] << " " << vNew[j] << " " << vOld[j] << endl;
    }
}


4 0 0 0 0
4 1 50 0 0
4 2 100 0 0
4 3 150 50 50
4 4 200 100 100


The next step is to setup a loop  to count backwards through time, and at each timestep another loop to go through all stock price values (except the boundaries). 
- We wish to use two time levels of storage for $V$, which we shall call `vNew` and `vOld`.
- Here let $V^i$ be represented by the vector `vNew`,
-  and $V^{i+1}$ be represented by the vector `vOld`.
- For an explicit method we have the formula
$$
V^i_j = \frac{1}{1+r\Delta t} \bigg( A V^{i+1}_{j+1} + B V^{i+1}_{j} + C V^{i+1}_{j-1}\bigg)
$$
where
$$
A=\left(\frac12 \sigma^2j^2 + \frac12 rj\right)\Delta t
$$
$$
B=1-\sigma^2j^2\Delta t
$$
$$
C=\left(\frac12 \sigma^2j^2 - \frac12 rj\right)\Delta t
$$
- For a call option, at the lower boundary we have
$$
V(S=0,t) = 0
$$
- For a call option at the upper boundary we have
$$
V(S=S_\text{max},t) = S_\text{max} - Xe^{-r(T-t)}
$$

In [3]:
{
  // declare and initialise Black Scholes parameters
  double S0=100.,X=100.,T=1.,r=0.06,sigma=0.2;
  // declare and initialise grid paramaters 
  int iMax=4,jMax=4;
  // declare and initialise local variables (ds,dt)
  double S_max=2*X;
  double dS=S_max/jMax;
  double dt=T/iMax;
  // create storage for the stock price and option price (old and new)
  vector<double> S(jMax+1),vOld(jMax+1),vNew(jMax+1);
  // setup and initialise the stock price 
  for(int j=0;j<=jMax;j++)
  {
    S[j] = j*dS;
  }
  // setup and initialise the final conditions on the option price 
  for(int j=0;j<=jMax;j++)
  {
    vOld[j] = max(S[j]-X,0.);
    vNew[j] = max(S[j]-X,0.);
    cout << iMax << " " << j << " " << S[j] << " " << vNew[j] << " " << vOld[j] << endl;
  }
  // loop through time levels, setting the option price at each grid point, and also on the boundaries
  for(int i=iMax-1;i>=0;i--)
  {
    // apply boundary condition at S=0
    vNew[0] = 0.;
    cout << i << " " << 0 << " " << S[0] << " " << vNew[0] << " " << vOld[0] << endl;
    for(int j=1;j<=jMax-1;j++)
    {
      double A,B,C;
      A=0.5*sigma*sigma*j*j*dt+0.5*r*j*dt;
      B=1.-sigma*sigma*j*j*dt;
      C=0.5*sigma*sigma*j*j*dt-0.5*r*j*dt;
      vNew[j] = 1./(1.+r*dt)*(A*vOld[j+1]+B*vOld[j]+C*vOld[j-1]);
      cout << i << " " << j << " " << S[j] << " " << vNew[j] << " " << vOld[j] << endl;
    }
    // apply boundary condition at S=S_max
    vNew[jMax] = S[jMax] - X*exp(-r*(T-i*dt));
    cout << i << " " << jMax << " " << S[jMax] << " " << vNew[jMax] << " " << vOld[jMax] << endl;
    // set old values to new
    vOld=vNew;
  }
}

4 0 0 0 0
4 1 50 0 0
4 2 100 0 0
4 3 150 50 50
4 4 200 100 100
3 0 0 0 0
3 1 50 0 0
3 2 100 1.72414 0
3 3 150 51.4778 50
3 4 200 101.489 100
2 0 0 0 0
2 1 50 0.0212332 0
2 2 100 3.40581 1.72414
2 3 150 52.94 51.4778
2 4 200 102.955 101.489
1 0 0 0 0
1 1 50 0.0626537 0.0212332
1 2 100 5.04688 3.40581
1 3 150 54.3858 52.94
1 4 200 104.4 102.955
0 0 0 0 0
0 1 50 0.123264 0.0626537
0 2 100 6.64908 5.04688
0 3 150 55.8144 54.3858
0 4 200 105.824 104.4


# Tasks

- Try varying the value of $iMax$ and $jMax$, what happens to the solution if $iMax$ is not very large compared to $jMax$?
- Check the stability condition holds, can you set an appropriate value of $iMax$ given an input value for $jMax$?
- Plot out the solution $V(S=X,t=0)$ for different values of $jMax$, what does it look like?
- Increase $jMax$ by multiples of 2 starting at $jMax=10$, can you estimate the convergence rate?
- Change $S_{\text{max}}$ to $5X$ or $10X$ -- what effect does this have on the results? 
- Compare the efficiency of the method to that of binomial trees.

In [4]:
{
  // declare and initialise Black Scholes parameters
  double S0=100.,X=100.,T=1.,r=0.06,sigma=0.2;
  // declare and initialise grid paramaters 
  int iMax=40,jMax=40;
  // declare and initialise local variables (ds,dt)
  double S_max=2*X;
  double dS=S_max/jMax;
  double dt=T/iMax;
  // create storage for the stock price and option price (old and new)
  vector<double> S(jMax+1),vOld(jMax+1),vNew(jMax+1);
  // setup and initialise the stock price 
  for(int j=0;j<=jMax;j++)
  {
    S[j] = j*dS;
  }
  // setup and initialise the final conditions on the option price 
  for(int j=0;j<=jMax;j++)
  {
    vOld[j] = max(S[j]-X,0.);
    vNew[j] = max(S[j]-X,0.);
  }
  // loop through time levels, setting the option price at each grid point, and also on the boundaries
  for(int i=iMax-1;i>=0;i--)
  {
    // apply boundary condition at S=0
    vNew[0] = 0.;
    for(int j=1;j<=jMax-1;j++)
    {
      double A,B,C;
      A=0.5*sigma*sigma*j*j*dt+0.5*r*j*dt;
      B=1.-sigma*sigma*j*j*dt;
      C=0.5*sigma*sigma*j*j*dt-0.5*r*j*dt;
      vNew[j] = 1./(1.+r*dt)*(A*vOld[j+1]+B*vOld[j]+C*vOld[j-1]);
    }
    // apply boundary condition at S=S_max
    vNew[jMax] = S[jMax] - X*exp(-r*(T-i*dt));
    // set old values to new
    vOld=vNew;
  }
  for(int j=0;j<=jMax;j++)
    cout << "V("<<S[j]<<")="<<vNew[j]<<endl;
}

V(0)=0
V(5)=2.05298e-17
V(10)=1.2199e-14
V(15)=2.2993e-12
V(20)=1.9952e-10
V(25)=9.43437e-09
V(30)=2.68428e-07
V(35)=4.91408e-06
V(40)=6.08617e-05
V(45)=0.000531306
V(50)=0.00338993
V(55)=0.0163562
V(60)=0.0616914
V(65)=0.18789
V(70)=0.476663
V(75)=1.0367
V(80)=1.98334
V(85)=3.41276
V(90)=5.38217
V(95)=7.90249
V(100)=10.9431
V(105)=14.4444
V(110)=18.3317
V(115)=22.5277
V(120)=26.9606
V(125)=31.5703
V(130)=36.3017
V(135)=41.1492
V(140)=45.9249
V(145)=51.3225
V(150)=54.4605
V(155)=65.6869
V(160)=51.449
V(165)=108.825
V(170)=-11.9568
V(175)=256.313
V(180)=-211.768
V(185)=504.924
V(190)=-346.321
V(195)=400.067
V(200)=105.824


By choosing `iMax`$=40$ and `jMax`$=40$ we see the results for large $S$ do not look very good. This is because the value of $B$ for large $S$ is negative causing unstable results. Fix this by choosing `iMax` according to the stability condition:
$$
\text{iMax} = 2T\sigma^2\text{jMax}^2
$$.

In [5]:
{
  // declare and initialise Black Scholes parameters
  double S0=100.,X=100.,T=1.,r=0.06,sigma=0.2;
  // declare and initialise grid paramaters 
  int jMax=40;
  
  // choose iMax according to stability condition
  int iMax = 2*T*sigma*sigma*jMax*jMax;

  // declare and initialise local variables (ds,dt)
  double S_max=2*X;
  double dS=S_max/jMax;
  double dt=T/iMax;
    
  // create storage for the stock price and option price (old and new)
  vector<double> S(jMax+1),vOld(jMax+1),vNew(jMax+1);
  // setup and initialise the stock price 
  for(int j=0;j<=jMax;j++)
  {
    S[j] = j*dS;
  }
  // setup and initialise the final conditions on the option price 
  for(int j=0;j<=jMax;j++)
  {
    vOld[j] = max(S[j]-X,0.);
    vNew[j] = max(S[j]-X,0.);
  }
  // loop through time levels, setting the option price at each grid point, and also on the boundaries
  for(int i=iMax-1;i>=0;i--)
  {
    // apply boundary condition at S=0
    vNew[0] = 0.;
    for(int j=1;j<=jMax-1;j++)
    {
      double A,B,C;
      A=0.5*sigma*sigma*j*j*dt+0.5*r*j*dt;
      B=1.-sigma*sigma*j*j*dt;
      C=0.5*sigma*sigma*j*j*dt-0.5*r*j*dt;
      vNew[j] = 1./(1.+r*dt)*(A*vOld[j+1]+B*vOld[j]+C*vOld[j-1]);
    }
    // apply boundary condition at S=S_max
    vNew[jMax] = S[jMax] - X*exp(-r*(T-i*dt));
    // set old values to new
    vOld=vNew;
  }
  for(int j=0;j<=jMax;j++)
    cout << "V("<<S[j]<<")="<<vNew[j]<<endl;
}

V(0)=0
V(5)=3.64424e-16
V(10)=1.37707e-13
V(15)=1.72044e-11
V(20)=1.03123e-09
V(25)=3.50989e-08
V(30)=7.49034e-07
V(35)=1.07153e-05
V(40)=0.000107977
V(45)=0.000797458
V(50)=0.00446539
V(55)=0.0195446
V(60)=0.0687929
V(65)=0.200025
V(70)=0.492804
V(75)=1.05339
V(80)=1.99614
V(85)=3.41843
V(90)=5.38
V(95)=7.89418
V(100)=10.9318
V(105)=14.4331
V(110)=18.3227
V(115)=22.5221
V(120)=26.9588
V(125)=31.5709
V(130)=36.3089
V(135)=41.1349
V(140)=46.021
V(145)=50.9475
V(150)=55.9005
V(155)=60.8709
V(160)=65.8523
V(165)=70.8408
V(170)=75.8337
V(175)=80.8294
V(180)=85.8268
V(185)=90.8253
V(190)=95.8244
V(195)=100.824
V(200)=105.824


# Interpolation

We show here a simple example of how to take the code detailed above and output a value for the option at 
a given stock price using linear interpolation. We use a style here which allows for a more generic higher order
Lagrange polynomial approximation to be derived.

- Assume that we have the set of points $S_j$ and $vNew_j$ and we wish to find the value of the interpolated function $V(S)$.
- The linear approximation says that for points $j*$ and $j*+1$ we should choose
$$ V(S) = \frac{S - S_{j*+1}}{S_{j*} - S_{j*+1}}vNew_{j*} + \frac{S - S_{j*}}{S_{j*+1} - S_{j*}}vNew_{j*+1}
$$

There are three stages:
1. Given the value $S_0$, find $j^*$ such that $S_0 \in [j*\Delta S,(j*+1)\Delta S]$
2. Evaluate the Lagrange polynomial at $S=S_0$
3. Test and debug the code!!! 

You can check that the polynomial above is the equation of a line passing through the points $(S_{j^*},vNew_{j^*})$ and $(S_{j^*+1},vNew_{j^*+1})$. Try different values of $S_0$ and check they look ok.

Once you are convinced the code is working move everything out into a function. You will now be able to pass in $S_0$ as
an argument to the function, and simply return the value from the interpolation.

In [6]:
double explicitCallOption(
    // Black Scholes parameters
    double S0,
    double X,
    double T,
    double r,
    double sigma,
    // grid paramaters 
    int jMax,
    int gridScale // here Smax = gridScale*X
  )
{
  
  // choose iMax according to stability condition
  int iMax = 2*T*sigma*sigma*jMax*jMax;

  // declare and initialise local variables (ds,dt)
  double S_max=gridScale*X;
  double dS=S_max/jMax;
  double dt=T/iMax;
    
  // create storage for the stock price and option price (old and new)
  vector<double> S(jMax+1),vOld(jMax+1),vNew(jMax+1);
  // setup and initialise the stock price 
  for(int j=0;j<=jMax;j++)
  {
    S[j] = j*dS;
  }
  // setup and initialise the final conditions on the option price 
  for(int j=0;j<=jMax;j++)
  {
    vOld[j] = max(S[j]-X,0.);
    vNew[j] = max(S[j]-X,0.);
  }
  // loop through time levels, setting the option price at each grid point, and also on the boundaries
  for(int i=iMax-1;i>=0;i--)
  {
    // apply boundary condition at S=0
    vNew[0] = 0.;
    for(int j=1;j<=jMax-1;j++)
    {
      double A,B,C;
      A=0.5*sigma*sigma*j*j*dt+0.5*r*j*dt;
      B=1.-sigma*sigma*j*j*dt;
      C=0.5*sigma*sigma*j*j*dt-0.5*r*j*dt;
      vNew[j] = 1./(1.+r*dt)*(A*vOld[j+1]+B*vOld[j]+C*vOld[j-1]);
    }
    // apply boundary condition at S=S_max
    vNew[jMax] = S[jMax] - X*exp(-r*(T-i*dt));
    // set old values to new
    vOld=vNew;
  }
    // get j* such that S_0 \in [ j*dS , (j*+1)dS ]
  int jstar;
  jstar = S0/dS;
  double sum=0.;
  // run 2 point Lagrange polynomial interpolation
  sum = sum + (S0 - S[jstar+1])/(S[jstar]-S[jstar+1])*vNew[jstar];
  sum = sum + (S0 - S[jstar])/(S[jstar+1]-S[jstar])*vNew[jstar+1];
  return sum;
}

In [7]:
{
  // declare and initialise Black Scholes parameters
  double S0=100.,X=100.,T=1.,r=0.06,sigma=0.2;
  // declare and initialise grid paramaters 
  int jMax=40;
  int gridScale=2;
  cout << explicitCallOption(S0,X,T,r,sigma,jMax,gridScale) << endl;
}


10.9318


Now we can look at the convergence rate for this method at $S_0=X$ so that the value coincides with a grid node and also at $S_0\ne X$ so we aren't on a grid point.

First let's try with $S=X$:

In [8]:
{
    // declare and initialise Black Scholes parameters
    double S0=100.,X=100.,T=1.,r=0.06,sigma=0.2;
    // declare and initialise grid paramaters 
    int gridScale=2;
    
    double valueOld=1.;
    double diffOld=1.;
    int k=2;
    cout << " jMax  | V_exp     | ratio  | rate c   \n";
    for(int i=1;i<=6;i++)
    {
        int jMax=10*pow(k,i);
        double value = explicitCallOption(S0,X,T,r,sigma,jMax,gridScale);
        double diff = value - valueOld;
        double R = diffOld/diff;
        double c = log(R)/log(k);
        cout << jMax << " | " << value;
        cout << " | " << R << " | " ;
        cout << c << "\n";
        valueOld=value;
        diffOld=diff;
    }
}

 jMax  | V_exp     | ratio  | rate c   
20 | 10.7505 | 0.102559 | -3.28548
40 | 10.9318 | 53.7896 | 5.74925
80 | 10.9752 | 4.17268 | 2.06097
160 | 10.986 | 4.03707 | 2.01331
320 | 10.9887 | 4.00899 | 2.00324
640 | 10.9893 | 4.00223 | 2.0008


Ok looks like second order convergence as expected. Now try $S = 78.4379 \ne X$:

In [9]:
{
    // declare and initialise Black Scholes parameters
    double S0=78.4379,X=100.,T=1.,r=0.06,sigma=0.2;
    // declare and initialise grid paramaters 
    int gridScale=2;
    
    double valueOld=1.;
    double diffOld=1.;
    int k=2;
    cout << " jMax  | V_exp     | ratio  | rate c   \n";
    for(int i=1;i<=6;i++)
    {
        int jMax=10*pow(k,i);
        double value = explicitCallOption(S0,X,T,r,sigma,jMax,gridScale);
        double diff = value - valueOld;
        double R = diffOld/diff;
        double c = log(R)/log(k);
        cout << jMax << " | " << value;
        cout << " | " << R << " | " ;
        cout << c << "\n";
        valueOld=value;
        diffOld=diff;
    }
}

 jMax  | V_exp     | ratio  | rate c   
20 | 1.69964 | 1.4293 | 0.515312
40 | 1.70161 | 356.276 | 8.47685
80 | 1.68415 | -0.112482 | -nan
160 | 1.67759 | 2.66153 | 1.41226
320 | 1.6769 | 9.56275 | 3.25742
640 | 1.67628 | 1.10325 | 0.141756


Ok this is not great -- maybe higher order interpolation would help?